In [1]:
%pwd

'/Users/apoorvgupta'

In [2]:
%cd Downloads

/Users/apoorvgupta/Downloads


In [4]:
import pandas as pd
stack_posts = pd.read_csv("stack_posts.csv")

In [5]:
stack_posts.head(5)

,id,title,body,tags,answer_count
0,20758373,Inversion with ordinal scale,<p>Is there any way to find inversion of ordin...,d3.js,8
1,20517541,Root path in XAMPP,"<p>I've had this problem for a while, and have...",php|html|apache|xampp,6
2,20669437,Adding multiple projects to solution,<p>This seems to be extremely annoying. I crea...,asp.net|visual-studio,6
3,20830353,How to make an element's content editable with...,<p>How would I make it so the content of an el...,javascript|jquery|html|contenteditable,6
4,20414027,Batch Script: Validate Date Input,<p>I have a batch file that I use to create ne...,windows|validation|date|batch-file,6


Creating a deepcopy of stack_posts :-

In [7]:
import numpy as np 
import copy
import os
import re

stack_posts2 = copy.deepcopy (stack_posts)
stack_posts2 = stack_posts2.dropna(axis=0)

In [8]:
import collections

stack_posts2.head(4)

,id,title,body,tags,answer_count
0,20758373,Inversion with ordinal scale,<p>Is there any way to find inversion of ordin...,d3.js,8
1,20517541,Root path in XAMPP,"<p>I've had this problem for a while, and have...",php|html|apache|xampp,6
2,20669437,Adding multiple projects to solution,<p>This seems to be extremely annoying. I crea...,asp.net|visual-studio,6
3,20830353,How to make an element's content editable with...,<p>How would I make it so the content of an el...,javascript|jquery|html|contenteditable,6


The folowing function deletes the html tags from the bodies and titles of the question posts. Applying that function to our body column.

In [9]:

def deleteHtml(body):
    regex = re.compile('<.*?>')
    return re.sub(regex, '', body)

stack_posts2['body'] = stack_posts2['body'].apply(deleteHtml)
stack_posts2.head(5)

,id,title,body,tags,answer_count
0,20758373,Inversion with ordinal scale,Is there any way to find inversion of ordinal ...,d3.js,8
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",php|html|apache|xampp,6
2,20669437,Adding multiple projects to solution,This seems to be extremely annoying. I create ...,asp.net|visual-studio,6
3,20830353,How to make an element's content editable with...,How would I make it so the content of an eleme...,javascript|jquery|html|contenteditable,6
4,20414027,Batch Script: Validate Date Input,I have a batch file that I use to create new p...,windows|validation|date|batch-file,6


The type of the 'tags' column is string. The vertical line symbol in these strings acts as a delimiter to separate tags from one another. The following lines of code read those individual tags and add them to a list in a seperate column called 'tagArr'.

In [15]:
numRows = stack_posts2.shape[0]

newCol = []
for j in range (numRows):
    tagsArr = []
    iniStr = ''
    for i in stack_posts['tags'][j]:
        if (i == '|'):
            tagsArr.append(iniStr)
            iniStr = ''
        else:
            iniStr = iniStr + i

    tagsArr.append(iniStr)
    newCol.append(tagsArr)

stack_posts2['tagsArr'] = newCol

The following explodes rows of the dataframe based on the concatenated tags column 'tags'.

In [56]:
split_tags_df = stack_posts2.assign(tags=stack_posts2['tags'].str.split('|')).explode('tags')
#
print (split_tags_df.shape)

split_tags_df.head(4)

(43636, 6)


,id,title,body,tags,answer_count,tagsArr
0,20758373,Inversion with ordinal scale,Is there any way to find inversion of ordinal ...,d3.js,8,[d3.js]
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",php,6,"[php, html, apache, xampp]"
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",html,6,"[php, html, apache, xampp]"
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",apache,6,"[php, html, apache, xampp]"


Converting the 'tags' column, which is currently of type string, to list of single string. We do this because the multiLabelBinarizer method needs a vector of labels (just some requisite data cleaning).

In [88]:
def Convert(string): 
    li = list(string.split("-")) 
    return li 


database = []
for i in split_tags_df.tags:
    myList = Convert(i)
    database.append(myList)
    
split_tags_df['Tags'] = database
    
split_tags_df.head(8)

43636


,id,title,body,tags,answer_count,tagsArr,Tags
0,20758373,Inversion with ordinal scale,Is there any way to find inversion of ordinal ...,d3.js,8,[d3.js],[d3.js]
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",php,6,"[php, html, apache, xampp]",[php]
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",html,6,"[php, html, apache, xampp]",[html]
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",apache,6,"[php, html, apache, xampp]",[apache]
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",xampp,6,"[php, html, apache, xampp]",[xampp]
2,20669437,Adding multiple projects to solution,This seems to be extremely annoying. I create ...,asp.net,6,"[asp.net, visual-studio]",[asp.net]
2,20669437,Adding multiple projects to solution,This seems to be extremely annoying. I create ...,visual-studio,6,"[asp.net, visual-studio]","[visual, studio]"
3,20830353,How to make an element's content editable with...,How would I make it so the content of an eleme...,javascript,6,"[javascript, jquery, html, contenteditable]",[javascript]


Resetting the index

In [104]:
split_tags_df = split_tags_df.reset_index(drop=True)
split_tags_df.head(8)

,id,title,body,tags,answer_count,tagsArr,Tags
0,20758373,Inversion with ordinal scale,Is there any way to find inversion of ordinal ...,d3.js,8,[d3.js],[d3.js]
1,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",php,6,"[php, html, apache, xampp]",[php]
2,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",html,6,"[php, html, apache, xampp]",[html]
3,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",apache,6,"[php, html, apache, xampp]",[apache]
4,20517541,Root path in XAMPP,"I've had this problem for a while, and have un...",xampp,6,"[php, html, apache, xampp]",[xampp]
5,20669437,Adding multiple projects to solution,This seems to be extremely annoying. I create ...,asp.net,6,"[asp.net, visual-studio]",[asp.net]
6,20669437,Adding multiple projects to solution,This seems to be extremely annoying. I create ...,visual-studio,6,"[asp.net, visual-studio]","[visual, studio]"
7,20830353,How to make an element's content editable with...,How would I make it so the content of an eleme...,javascript,6,"[javascript, jquery, html, contenteditable]",[javascript]


Importing some libraries beforehand.

In [24]:
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

Creating a multi-label binarizer instance. Using that instance to create a binary matrix through the transform method. I am using a binary matrix because it is simplest to implement understand for a beginner like me, but using an adapted algorithm like MLkNN would probably have been better for this problem.

In [110]:
splitTags2 = split_tags_df.drop(columns = ['tags', 'answer_count', 'tagsArr'])

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(splitTags2.Tags)
labels = multilabel_binarizer.classes_

labels

array(['.doc', '.htaccess', '.net', ..., 'ztree', 'zurb', 'zxing'],
      dtype=object)

Splitting my df into test and train dataframes.

In [111]:
numRowsUsed = int (splitTags2.shape[0] * 0.75) # not using all data (only using 75%), just in case more unseen data is required in the future.
train,test=train_test_split(splitTags2[:numRowsUsed],test_size=0.25,random_state=16)

Checking the shape to make sure the split happened correctly.

In [112]:
train.shape,test.shape

((24545, 4), (8182, 4))

Parsing columns.

In [113]:
X_train_t=train['title']
X_train_b=train['body']
X_test_t=test['title']
X_test_b=test['body']

Finally transforming our tags column using the transform method from our multi-lable binarizer instance.

In [128]:
from IPython.utils import io

# using io to suppress output
with io.capture_output() as captured:
    y_test=multilabel_binarizer.transform(test['Tags'])
    y_train=multilabel_binarizer.transform(train['Tags'])

In [125]:
import nltk
nltk.download('punkt') # required for tokenization

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/apoorvgupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Operating on title testing and training dataframes.
Note: Later, we'll repeat the same for 'body' dataframes.

Tokenizing the titles and finding the greatest length of token array in the training dataframe's title column.

In [142]:
len_of_title_tokens = []
for x in train['title']:
    len_of_title_tokens.append(len(word_tokenize(x)))

    
print (max(len_of_title_tokens))

(24545, 4)

Find the 97th quartile length

In [143]:
np.quantile(len_of_title_tokens,0.97)

19.0

Disregarding the absurdly long titles and transforming each token text value to sequence of integeres using the texts_to_sequences method. 

In [183]:
max_len_of_title_tokens = 19
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_t)
sequences_train_t = tok.texts_to_sequences(X_train_t)

Finding number of unique tokens.

In [184]:
vocab_len_t=len(tok.index_word.keys())
print(vocab_len_t)

10288


Padding the sequences with 0 until all the sequences have the same length using Keras' pad_sequences method.

In [185]:
sequences_matrix_train_t = sequence.pad_sequences(
    sequences_train_t,maxlen = max_len_of_title_tokens)

print (sequences_matrix_train_t.shape)

(24545, 19)


Repeating the last two steps for testing data (prev 2 were for training data)

In [186]:
sequences_test_t = tok.texts_to_sequences(X_test_t)
sequences_matrix_test_t = sequence.pad_sequences(
    sequences_test_t,maxlen = max_len_of_title_tokens)

In [187]:
print (sequences_matrix_train_t.shape)
print (sequences_matrix_test_t.shape)
print (y_train.shape)
print (y_test.shape)

(24545, 19)
(8182, 19)
(24545, 4289)
(8182, 4289)


# Operating on 'body' testing and training dataframes.
Excluding markdown description cells since these operations are the same.

In [188]:
len_of_body_tokens=[]
for sent in train['body']:
    len_of_body_tokens.append(len(word_tokenize(sent)))
    
print (max(len_of_body_tokens))

6800


In [189]:
print (np.quantile(len_of_body_tokens,0.931))

676.0


In [190]:
max_len_of_body_tokens = 676
tok = Tokenizer(char_level = False, split = ' ')
tok.fit_on_texts(X_train_b)
sequences_train_b = tok.texts_to_sequences(X_train_b)

In [191]:
vocab_len_b = len(tok.index_word.keys())
print (vocab_len_b)

82808


In [192]:
sequences_matrix_train_b = sequence.pad_sequences(
    sequences_train_b, maxlen = max_len_of_body_tokens)

print (sequences_matrix_train_b.shape)

(24545, 676)


In [193]:
sequences_test_b = tok.texts_to_sequences(X_test_b)
sequences_matrix_test_b = sequence.pad_sequences(
    sequences_test_b, maxlen = max_len_of_body_tokens)

In [194]:
print (
    sequences_matrix_train_t.shape,
    sequences_matrix_train_b.shape,
    y_train.shape)

(24545, 19) (24545, 676) (24545, 4289)


In [195]:
print (
    sequences_matrix_test_t.shape,
    sequences_matrix_test_b.shape,
    y_test.shape)

(8182, 19) (8182, 676) (8182, 4289)


Next, I built a recurrent neural network.
1. For title column, I used an embedding layer with input output as vocab length + 1, 2000 respectively. I also used a GRU layer and 1 densed output layer with shape 10.
2. For body column, I used a similar embedding layer except with output 170 and a GRU layer.
3. Next, I combined these two.
4. Lastly, I added the following in order - a dense layer, dropout layer, batchNormalization layer, dense output layer

In [222]:
def RNN():
    
    # RNN part for the title column
    title_input = Input(name='title_input',shape=[max_len_of_title_tokens])
    title_Embed = Embedding(vocab_len_t+1,2000,input_length=max_len_of_title_tokens,mask_zero=True,name='title_Embed')(title_input)
    gru_out_t = GRU(300)(title_Embed)
    
    # Using dense method for auxillary output to smoothen the effects of the GRU
    auxiliary_output = Dense(10, activation='sigmoid', name='aux_output')(gru_out_t)   
    
    # RNN part for the body column
    body_input = Input(name='body_input',shape=[max_len_of_body_tokens]) 
    body_Embed = Embedding(vocab_len_b+1,170,input_length=max_len_of_body_tokens,mask_zero=True,name='body_Embed')(body_input)
    gru_out_b = GRU(200)(body_Embed)
    
    # combining these two
    combined = concatenate([gru_out_t, gru_out_b])
    
    # Feeding combined to our remaining two dense layers
    dense1 = Dense(400,activation='relu')(combined)
    dropout = Dropout(0.5)(dense1)
    batchNormalizer = BatchNormalization()(dropout) 
    dense2 = Dense(150,activation='relu')(batchNormalizer)
    
    main_output = Dense(10, activation='sigmoid', name='main_output')(dense2)
    
    model = Model(inputs=[title_input, body_input],outputs=[main_output, auxiliary_output])
    
    return model

In [223]:
model = RNN()
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_input (InputLayer)        [(None, 19)]         0                                            
__________________________________________________________________________________________________
body_input (InputLayer)         [(None, 676)]        0                                            
__________________________________________________________________________________________________
title_Embed (Embedding)         (None, 19, 2000)     20578000    title_input[0][0]                
__________________________________________________________________________________________________
body_Embed (Embedding)          (None, 676, 170)     14077530    body_input[0][0]                 
_______________________________________________________________________________________

We can see that the idea of cross-entropy may be useful for optimizing a classification model. Cross-entropy is widely used as a loss function when optimizing classification models. Using cross-entropy gives faster training and improved generalization over sum of squares error.

Each example has a known class label with a probability of 1.0, and a probability of 0.0 for all other labels (my sequences matrices are one-hot). My model can estimate the probability of an example belonging to each class label. Cross-entropy can then be used to calculate the difference between the two probability distributions.

My model seeks to approximate the target probability distribution, i.e. my prediction for the classification of the post. For this, I need to account for the loss; the following compilation does exactly that.

In [237]:
import tensorflow as tf

model.compile(optimizer='adam',loss={'main_output': 'categorical_crossentropy', 'aux_output': 'categorical_crossentropy'},
              metrics=['accuracy'])
model.run_eagerly = True

Applying model over validation data.

In [239]:
tf.print (model.fit({'title_input': sequences_matrix_train_t, 'body_input': sequences_matrix_train_b},
          {'main_output': y_train, 'aux_output': y_train},
          validation_data=[{'title_input': sequences_matrix_test_t, 'body_input': sequences_matrix_test_b},
          {'main_output': y_test, 'aux_output': y_test}],
          epochs=5, batch_size=200)
         )


Epoch 1/5


ValueError: Shapes (200, 4289) and (200, 10) are incompatible

Predicting...

In [ ]:
(predicted_main, predicted_aux)=model.predict({'title_input': sequences_matrix_test_t, 'body_input': sequences_matrix_test_b},verbose=1)

Viewing F1 score

In [ ]:
from sklearn.metrics import classification_report,f1_score

print(f1_score(y_test,predicted_main>.55,average='samples'))

Viewing classification report:-

In [ ]:
print(classification_report(y_test,predicted_main>.55))

Other results

In [ ]:
test.iloc[17] #random result

In [ ]:
predicted_main[17].round(decimals = 2)

In [ ]:
print (labels)